In [2]:
%pip install -U spacy
!python -m spacy download en_core_web_lg

Note: you may need to restart the kernel to use updated packages.
/Users/thomasheigl/.pyenv/versions/3.10.6/envs/smart_food_diary/lib/python3.10/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 5.1 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [2]:
import spacy

nlp = spacy.load('en_core_web_lg')

/Users/thomasheigl/.pyenv/versions/3.10.6/envs/smart_food_diary/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/thomasheigl/.pyenv/versions/3.10.6/envs/smart_food_diary/lib/python3.10/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/Users/thomasheigl/.pyenv/versions/3.10.6/envs/smart_food_diary/lib/python3.10/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [3]:
nlp.vocab['Burger'].similarity(nlp.vocab['stone'])

-0.002288245363160968

In [89]:
nlp.vocab['Burger'].similarity(nlp.vocab['sandwich'])

0.45575466752052307

In [5]:
nlp.vocab['salad'].similarity(nlp.vocab['tomato'])

0.6763531565666199

In [11]:
nlp.vocab['Burger'].similarity(nlp.vocab['McDonalds McFilet'])

/var/folders/yn/kx4gh2414_q2phcdl6vwqdh40000gn/T/ipykernel_20115/1509192265.py:1: UserWarning: [W008] Evaluating Lexeme.similarity based on empty vectors.
  nlp.vocab['Burger'].similarity(nlp.vocab['McDonalds McFilet'])


0.0

## Problem: Spacy won't find a vector for words that are not in the dict yet

# Here's a non AI approach

In [78]:
import pandas as pd

food_names = pd.read_csv('../data/food_names.csv')['title'].str.lower().to_list()

In [85]:
import difflib
from difflib import SequenceMatcher

def closest_matches(meal_name: str):
    def similar(a, b):
        return SequenceMatcher(None, a, b).ratio()

    matches = difflib.get_close_matches(meal_name, food_names)

    if len(matches) == 0:
        return None

    return [{ 'title': x, 'similarity': similar(x, meal_name) } for x in matches]

#[{ 'title': x, 'similarity': similar(x, meal_name) } for x in matches]
closest_matches("mashed potatoes and fish fingers")

[{'title': 'mashed potatoes with cheddar', 'similarity': 0.7},
 {'title': 'mashed potatoes, baby reds', 'similarity': 0.6896551724137931},
 {'title': 'mashed potatoes, instant', 'similarity': 0.6785714285714286}]

# Teasting on all food from the food diaries

In [80]:
import json

def load_data(path):
    with open(path, 'r') as f:
        data = json.load(f)
    return data

json = load_data("../data/food-diary-examples-classified.json")
data = [x for x in json['annotations'] if x is not None]

# Read the food out of the json data
diary_entries = []
for x in data:
    title = x[0]
    entities = x[1]['entities']

    relevant = [(y[0], y[1]) for y in entities if y[2] in ['FOOD', 'LIQUID']]
    meal_names = [title[x[0]:x[1]].lower() for x in relevant]

    diary_entries.extend(meal_names)

diary_entries = list(set(diary_entries))

In [82]:
[(meal_name, closest_matches(meal_name)) for meal_name in diary_entries[:15]]

[('veganz protein choc bar chocolate brownie style',
  [{'title': 'protein plus bar, chocolate brownie',
    'similarity': 0.7317073170731707},
   {'title': 'protein plus bar, chocolate brownie',
    'similarity': 0.7317073170731707},
   {'title': 'lean protein & fiber bar, chocolate almond brownie',
    'similarity': 0.7010309278350515}]),
 ('egg',
  [{'title': 'egg', 'similarity': 1.0},
   {'title': 'egg', 'similarity': 1.0},
   {'title': 'eggs', 'similarity': 0.8571428571428571}]),
 ('spaghetti pizza with cheese',
  [{'title': 'spaghetti sauce, three cheese', 'similarity': 0.75},
   {'title': 'organic spaghetti with cheese', 'similarity': 0.75},
   {'title': 'spam with cheese', 'similarity': 0.6976744186046512}]),
 ('fried egg',
  [{'title': 'fried eggplant', 'similarity': 0.782608695652174},
   {'title': 'gummy fried eggs', 'similarity': 0.72},
   {'title': 'gummi fried eggs', 'similarity': 0.72}]),
 ('cherry peppers with cream cheese filling',
  [{'title': 'cupcakes with cream che